In [61]:
import pandas as pd

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
train_data.shape

(3000888, 6)

In [62]:
pd.get_dummies(train_data, columns=['family'])

,id,date,store_nbr,sales,onpromotion,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,family_BOOKS,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,0,2013-01-01,1,0.000,0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,2013-01-01,1,0.000,0,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2,2013-01-01,1,0.000,0,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,2013-01-01,1,0.000,0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,4,2013-01-01,1,0.000,0,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,438.133,0,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3000884,3000884,2017-08-15,9,154.553,1,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3000885,3000885,2017-08-15,9,2419.729,148,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3000886,3000886,2017-08-15,9,121.000,8,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [63]:
stores_df = pd.read_csv('data/stores.csv')
stores_df_transformed = stores_df.drop(['city', 'state', 'type'], axis=1)
merged_store_train = pd.merge(train_data, stores_df_transformed, on=['store_nbr','store_nbr'])

In [64]:
merged_store_train = merged_store_train.drop(['store_nbr'], axis=1)

In [65]:
merged_store_train.head()

,id,date,family,sales,onpromotion,cluster
0,0,2013-01-01,AUTOMOTIVE,0.0,0,13
1,1,2013-01-01,BABY CARE,0.0,0,13
2,2,2013-01-01,BEAUTY,0.0,0,13
3,3,2013-01-01,BEVERAGES,0.0,0,13
4,4,2013-01-01,BOOKS,0.0,0,13


In [66]:
holidays_df = pd.read_csv('data/holidays_events.csv')

mask = holidays_df['transferred'].apply(lambda x: x is False)
holidays_filtered = holidays_df[mask].copy()
holidays_filtered['is_holiday'] = True
holidays_filtered = holidays_filtered.drop(['type', 'locale', 'locale_name', 'description', 'transferred'], axis=1)
holidays_filtered.head()

,date,is_holiday
0,2012-03-02,True
1,2012-04-01,True
2,2012-04-12,True
3,2012-04-14,True
4,2012-04-21,True


In [67]:
merged_store_holiday_train = pd.merge(merged_store_train, holidays_filtered, on=['date', 'date'], how='left')

In [68]:
merged_store_holiday_train.is_holiday = merged_store_holiday_train.is_holiday.fillna(False)

In [75]:
oil_df = pd.read_csv('data/oil.csv')
oil_df.columns

Index(['date', 'dcoilwtico'], dtype='object')

In [78]:
filled_oil_df = oil_df.copy()
filled_oil_df['oil_price'] = oil_df.dcoilwtico.ffill().bfill()

In [79]:
filled_oil_df = filled_oil_df.drop(['dcoilwtico'], axis=1)

In [80]:
merged_store_holiday_train.columns

Index(['id', 'date', 'family', 'sales', 'onpromotion', 'cluster',
       'is_holiday'],
      dtype='object')

In [81]:
final_merged_df = merged_store_holiday_train = pd.merge(merged_store_holiday_train, filled_oil_df, on=['date', 'date'])

In [82]:
final_merged_df.head()

,id,date,family,sales,onpromotion,cluster,is_holiday,oil_price
0,0,2013-01-01,AUTOMOTIVE,0.0,0,13,True,93.14
1,1,2013-01-01,BABY CARE,0.0,0,13,True,93.14
2,2,2013-01-01,BEAUTY,0.0,0,13,True,93.14
3,3,2013-01-01,BEVERAGES,0.0,0,13,True,93.14
4,4,2013-01-01,BOOKS,0.0,0,13,True,93.14


In [84]:
df_with_dummies = pd.get_dummies(final_merged_df, columns=['family'])

In [85]:
df_with_dummies.head()

,id,date,sales,onpromotion,cluster,is_holiday,oil_price,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,0,2013-01-01,0.0,0,13,True,93.14,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,2013-01-01,0.0,0,13,True,93.14,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,2,2013-01-01,0.0,0,13,True,93.14,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,3,2013-01-01,0.0,0,13,True,93.14,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4,2013-01-01,0.0,0,13,True,93.14,False,False,False,...,False,False,False,False,False,False,False,False,False,False
